### Titanic Dataset Cleaning Project

#### This project uses the Titanic dataset from Kaggle. The aim is to clean this dataset and transform it into a more usable dataset, potentially for EDA (exploratory data analysis).

#### Objectives

- Take a quick look at the dataset, understanding the columns and the data contained within them.
- Check for inaccurate, inconsistent, or irrelevant data
- Fix this by handling missing values, removing duplicates, fixing data types and handling outliers
- Also create new 'features' (columns) useful for analysis:
    - 'Title' from passenger names
    - 'FamilySize' combining siblings/spouses and parents/children
    - 'IsAlone' for passengers travelling alone (FamilySize being 1)

#### Data Source

The dataset comes from Kaggle: [Titanic - Machine Learning from Disaster](https://www.kaggle.com/c/titanic/data).

- `train.csv` is used for cleaning and feature engineering.
- Original dataset has 891 rows and 12 columns; after cleaning, we have 14 columns with new features.


#### Importing the Libraries, Opening the CSV and Quick Look

In [71]:
# start by importing both pandas and numpy

import pandas as pd
import numpy as np

# load the train datafile within the titanic folder into the notebook

df = pd.read_csv('Desktop/Pandas practice/titanic/train.csv')

# get a quick preview of the dataset (first five rows)

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### Quick Overview of Basic Info

In [72]:
# use shape to get the total number of rows and columns

print("shape: ", df.shape)

# use info() to see the names of the columns, associated datatypes as well as null counts for each

df.info()

# use describe() to provide a stastitical summary of all the columns
# all columns are included, even non-numerical ones

df.describe(include='all')

shape:  (891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


#### Check for Missing Values

In [73]:
# Check for missing values within each column
# This will sum the number of missing values within each column

df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

#### Handle the Missing Values (Dropping Columns)

In [74]:
# Decided to drop the Column 'Cabin' as there are too many missing values

df = df.drop(columns=['Cabin'])

# Information to remind that the cabin column has been dropped

print('Cabin column dropped')

# Quick look at which columns still remain

print('Remaining columns:', df.columns.tolist())

Cabin column dropped
Remaining columns: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked']


#### Handle the Missing Values (Filling Columns)

In [75]:
# Will use the median in place of null values for the age column due to the mean being skewed on account of missing values

df['Age'] = df['Age'].fillna(df['Age'].median())

# Then will fill the 'Embarked' column with the most common value
# Bare in mind, the describe function above gave 'S' as the mode so this will replace any null values

df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Then check again for null values in the dataframe

df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

#### Checking and Removing Duplicates

In [76]:
# First check for duplicate rows within the dataset

print('Duplicates before:', df.duplicated().sum())

# Seems to be no duplicates but if there was, use below to remove said duplicates

df = df.drop_duplicates()

Duplicates before: 0


#### Fixing Data Types

In [77]:
# Seems as if Survived, Pclass and Embarked are integers when they should be categories
# Need to change the data type

df['Survived'] = df['Survived'].astype('category')
df['Pclass'] = df['Pclass'].astype('category')
df['Embarked'] = df['Embarked'].astype('category')

# Also work converting both Ticket and Name to strings 

df['Ticket'] = df['Ticket'].astype('string')
df['Name'] = df['Name'].astype('string')

# Once again, display the data types

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    category
 2   Pclass       891 non-null    category
 3   Name         891 non-null    string  
 4   Sex          891 non-null    object  
 5   Age          891 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    string  
 9   Fare         891 non-null    float64 
 10  Embarked     891 non-null    category
dtypes: category(3), float64(2), int64(3), object(1), string(2)
memory usage: 58.8+ KB


#### Cleaning Text Data

In [78]:
# Might also be worth converting the Sex to a category dtype but first need to standardize it
# This changes each value to lower cases and removes leading and trailing spaces etc

df['Sex'] = df['Sex'].str.lower().str.strip()

# Now convert to a category

df['Sex'] = df['Sex'].astype('category')

# Also clean the column Name for leading and trailing spaces

df['Name'] = df['Name'].str.strip()

#### Creating New Useful Columns

In [79]:
# Can extract the passenger title (Mr, Mrs etc.) from the Name column

# First will need to split the string at the comma and get the part after
df['Title'] = df['Name'].str.split(",").str[1]
# Then split it by the period to get the part before
df['Title'] = df['Title'].str.split(".").str[0]
# Finally remove any potential whitespace
df['Title'] = df['Title'].str.strip()


# Creating a value called 'Rare' for rare titles

# First create a counts variable which provides the value counts of each Title (Mrs etc.)
counts = df['Title'].value_counts()
# Then create a rare titles variable and use boolean indexing to get the titles which are less common than 10 instances
# The ".index" helps to return the title names as opposed to just the counts
rare_titles = counts[counts < 10].index
# Finally, replace the current rare titles with "Rare" to simplify the dataset
df['Title'] = df['Title'].replace(rare_titles, "Rare")

# Creating a column called FamilySize to the number of family members on board
# Combine the SibSp column and the Parch column then add 1 to include the individual themselves
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1


# Creating a column called IsAlone (assuming a family size of 1 means they are alone)
# Returns a boolean of True or False which is converted into an integer yielding 1 (True) and 0 (False)
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

# Look at the first 5 rows for the new columns
df[['Title', 'FamilySize', 'IsAlone']].head()

,Title,FamilySize,IsAlone
0,Mr,2,0
1,Mrs,2,0
2,Miss,1,1
3,Mrs,2,0
4,Mr,1,1


#### Handling Outliers

In [80]:
# Check the stats on the Fare column to see if there are any outliers

df['Fare'].describe()
# The max is far from the median so may be an outlier

# Need to reduce the impact of the outlier by using a cap (at the 99th percentile)

fare_cap = df['Fare'].quantile(0.99)

# Need to make any values above the fare_cap equal to the fare_Cap

df['Fare']= df['Fare'].clip(upper=fare_cap)

df['Fare'].describe()

count    891.000000
mean      31.224767
std       42.524125
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      249.006220
Name: Fare, dtype: float64

#### Final Checks

In [81]:
# Check if the number of columns to see if the new ones have been added

print(df.shape)

# Check to see if there are still any null values in any column

print(df.isnull().sum())

# Confirm that the datatypes have been changed etc.

df.info()

# Display the first 5 rows, just for a quick overview of the cleaned data

df.head()

(891, 14)
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
Title          0
FamilySize     0
IsAlone        0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    category
 2   Pclass       891 non-null    category
 3   Name         891 non-null    string  
 4   Sex          891 non-null    category
 5   Age          891 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    string  
 9   Fare         891 non-null    float64 
 10  Embarked     891 non-null    category
 11  Title        891 non-null    object  
 12  FamilySize   891 non-null  

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title,FamilySize,IsAlone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,Mr,2,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,Mrs,2,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,Miss,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,Mrs,2,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,Mr,1,1


#### Save the Cleaned Dataset as a CSV

In [82]:
# Finally, save the clean dataset

df.to_csv('Desktop/Pandas practice/titanic/titanic_clean.csv', index=False)

print('Dataset is clean and has now been saved!')

Dataset is clean and has now been saved!
